In [ ]:
!pip install hdfs

In [ ]:
from cdx.cdx_helper import cdx_query
from hdfs import InsecureClient
import zlib

def stream_gzip_decompress(stream):
    dec = zlib.decompressobj(32 + zlib.MAX_WBITS)  # offset 32 to skip the header
    for chunk in stream:
        rv = dec.decompress(chunk)
        if rv:
            yield rv

client = InsecureClient('http://hdfs.api.wa.bl.uk:80', user='access')

for h in cdx_query("http://www.jisc.ac.uk/whatwedo/programmes/programme_preservation/2008sigprops.aspx", limit=100):
    print(h.filename, h.offset, h.length)
    with client.read(h.filename, offset=h.offset, chunk_size=16*1024) as reader:
        for unc in stream_gzip_decompress(reader):
            print(unc.decode('utf-8'))
            break
    
